### Import the necessary library

In [2]:
import torch.utils.data
import scanpy as sc
from sklearn.cluster import KMeans
from scipy.spatial import distance
import scipy.sparse
import scib 
import numpy as np
import random
import pandas as pd
from torch.distributions import  kl_divergence as kl
torch.backends.cudnn.benchmark = True
from sklearn.neighbors import KDTree
from sklearn import preprocessing
import scipy
import scib

In [3]:
import scanpy as sc
import numpy as np
import pandas as pd
import time
import scvi
import anndata
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix
import umap
import matplotlib.pyplot as plt
import os
import tempfile
import seaborn as sns
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.12/site-packages/lightning/fabric/__init__.py:40: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
/opt/anaconda3/lib/python3.12/site-packages/pkg_resources/__init__.py:2317: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/__init__.py:37: Deprecated call to `pkg_resources.declare_namespace('lightning.pytorch')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to 

In [4]:
import warnings

sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

import warnings
warnings.filterwarnings("ignore")

# Suppress specific ImportWarning
warnings.filterwarnings("ignore", category=ImportWarning, message=".*AltairImportHook.find_spec() not found; falling back to find_module.*")

In [5]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

#Perhaps require the library relocation (to the library contains kBET and lisi library)
robjects.r('.libPaths(c("current path", "library location"))')

rscript = '''
library(kBET)
# library(lisi)
'''
robjects.r(rscript)
kbet = robjects.r('kBET')
# lisi = robjects.r['compute_lisi']

### read the data

In [6]:
# load the prediction results to adata
import anndata
adata = anndata.read_h5ad("../human_immune.h5ad")
adata

AnnData object with n_obs × n_vars = 33506 × 12303
    obs: 'batch', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'study', 'tissue'
    layers: 'counts'

In [7]:
adata.obs['cell_type'] = adata.obs['final_annotation']
adata

AnnData object with n_obs × n_vars = 33506 × 12303
    obs: 'batch', 'chemistry', 'data_type', 'dpt_pseudotime', 'final_annotation', 'mt_frac', 'n_counts', 'n_genes', 'sample_ID', 'size_factors', 'species', 'study', 'tissue', 'cell_type'
    layers: 'counts'

## Print the normal metrics (for scib)

In [19]:
import pandas as pd
import scib

def calculate_metrics_for_embeddings(file_path, adata, embedding_keys, batch_key='batch', label_key='cell_type'):
    # Read the CSV file into a DataFrame
    combined_embeddings = pd.read_csv(file_path, index_col=0)

    # Ensure indices match
    combined_embeddings = combined_embeddings.loc[adata.obs_names]

    # Split the DataFrame into separate DataFrames for each embedding
    embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}

    # Assign the embeddings back to adata.obsm
    for key, df in embeddings_dict.items():
        adata.obsm[key] = df.values

    # Initialize an empty list to store results
    results = []

    # Loop through each embedding and calculate metrics
    for key in embedding_keys:
        print(f"File: {file_path}, Embedding: {key}")
        sc.pp.neighbors(adata, use_rep=key)
        scib.me.cluster_optimal_resolution(adata, cluster_key="cluster", label_key=label_key)
        metrics = scib.me.metrics(
            adata, adata_int=adata, 
            silhouette_=True, graph_conn_=True, 
            ari_=True, nmi_=True, isolated_labels_=True, 
            isolated_labels_f1_=True,
            isolated_labels_asw_=True, 
            batch_key=batch_key, label_key=label_key, 
            embed=key
        )
        
        for metric, score in metrics.items():
            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'metric': metric,
                'score': score
            }
            results.append(result_entry)
            
            # Print the file path, embedding key, and result
            print(f"File: {file_path}, Embedding: {key}, Metric: {metric}, Score: {score}")

    return results

In [20]:
file_paths = [
    # "../embeddings/full_annotated_supervised_human_immune_new.csv",
    # "../embeddings/cell_assign_human_immune_new.csv",
    # "../embeddings/azimuth_human_immune_new.csv",
    # "../embeddings/singleR_human_immune_new.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_30.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_50.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_70.csv",
    # # "../embeddings/mixing_at_edge_human_immune_new_30.csv",
    # # "../embeddings/mixing_at_edge_human_immune_new_50.csv",
    # # "../embeddings/mixing_at_edge_human_immune_new_70.csv",
    # "../embeddings/randomly_missing_human_immune_new_30.csv",
    # "../embeddings/randomly_missing_human_immune_new_50.csv",
    # "../embeddings/randomly_missing_human_immune_new_70.csv"
    "../embeddings/azimuth_fuzzy_human_immune_new.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_ItClust"]

# Prepare a list to collect all results
all_results = []

# Loop through each file and calculate metrics
for file_path in file_paths:
    results = calculate_metrics_for_embeddings(file_path, adata, embedding_keys)
    all_results.extend(results)

# Now `all_results` contains metrics for all embedding files with identifying information

File: ../embeddings/azimuth_fuzzy_human_immune_new.csv, Embedding: X_ItClust
resolution: 0.1, nmi: 0.6977132505438458
resolution: 0.2, nmi: 0.6684224338697007
resolution: 0.3, nmi: 0.6529505798986721
resolution: 0.4, nmi: 0.6371982126845426
resolution: 0.5, nmi: 0.6372043496208533
resolution: 0.6, nmi: 0.6197714981145266
resolution: 0.7, nmi: 0.6058009011874619
resolution: 0.8, nmi: 0.5955466109491389
resolution: 0.9, nmi: 0.5890658574879557
resolution: 1.0, nmi: 0.5703364779945703
resolution: 1.1, nmi: 0.5718204378684432
resolution: 1.2, nmi: 0.5593031802160191
resolution: 1.3, nmi: 0.5625038688008644
resolution: 1.4, nmi: 0.5619274355985673
resolution: 1.5, nmi: 0.5503985706423716
resolution: 1.6, nmi: 0.5452187451428522
resolution: 1.7, nmi: 0.5435310949694687
resolution: 1.8, nmi: 0.5438221983316244
resolution: 1.9, nmi: 0.537622253055581
resolution: 2.0, nmi: 0.5350021406766218
optimised clustering against cell_type
optimal cluster resolution: 0.1
optimal score: 0.6977132505438458

In [21]:
# Flatten the nested dictionaries and extract desired scores
results = []
for entry in all_results:
    file_path = entry['file_path']
    embedding_key = entry['embedding_key']
    score_series = entry['score']
    for metric, value in score_series.items():
        # Create an entry for each score
        results.append({
            'file_path': file_path,
            'embedding_key': embedding_key,
            'metric': metric,
            'score': value
        })

# Convert results list to DataFrame
results_df = pd.DataFrame(results)

In [22]:
results_df

,file_path,embedding_key,metric,score
0,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,NMI_cluster/label,0.721547
1,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,ARI_cluster/label,0.651428
2,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,ASW_label,0.765234
3,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,ASW_label/batch,0.513863
4,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,PCR_batch,NaN
5,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,cell_cycle_conservation,NaN
6,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,isolated_label_F1,0.417323
7,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,isolated_label_silhouette,0.263762
8,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,graph_conn,0.817864
9,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,kBET,NaN


In [11]:
results_df.to_csv('../metrics/human_immune_scib_metrics_new_part1.csv', index=True)

## Print the overall metrics for TPT

In [23]:
# Positive and true positive cells defined in iMAP
def positive_true_positive(adata, batch_key='batch', celltype_key='cell_type', use_raw=False,k1=20, k2=100, tp_thr=3., distance='cosine', embed='X_pca'):
    celltype_list = adata.obs[celltype_key]
    batch_list = adata.obs[batch_key]

    temp_c = adata.obs[celltype_key].value_counts()
    temp_b = pd.crosstab(adata.obs[celltype_key], adata.obs[batch_key])
    temp_b_prob = temp_b.divide(temp_b.sum(1), axis=0)
    
    if use_raw:
        if isinstance(adata.X, scipy.sparse.csr.csr_matrix):
            X = adata.X.todense()
        else:
            X = adata.X
    else:
        X = adata.obsm[embed]
    if distance == 'cosine':
        X = preprocessing.normalize(X, axis=1)

    t1 = KDTree(X)

    p_list = []
    tp_list = []

    for cell in range(len(X)):

        # Discriminate positive cells
        neig1 = min(k1, temp_c[celltype_list[cell]])
        NNs = t1.query(X[cell].reshape(1,-1), neig1+1, return_distance=False)[0, 1:]
        c_NN = celltype_list[NNs]
        true_rate = sum(c_NN == celltype_list[cell])/neig1
        if true_rate > 0.5:
            p_list.append(True)
        else:
            p_list.append(False)

        # Discriminate true positive cells
        if p_list[cell] == True:
            neig2 = min(k2, temp_c[celltype_list[cell]])
            NNs = t1.query(X[cell].reshape(1,-1), neig2, return_distance=False)[0]
            NNs_c = celltype_list[NNs]
            NNs_i = NNs_c == celltype_list[cell]
            NNs = NNs[NNs_i] # get local neighbors that are from the same cell type
            neig2 = len(NNs)
            NNs_b = batch_list[NNs]

            max_b = 0
            b_prob = temp_b_prob.loc[celltype_list[cell]]
            for b in set(batch_list):
                if b_prob[b] > 0 and b_prob[b] < 1:
                    p_b = sum(NNs_b == b)
                    stat_b = abs(p_b - neig2*b_prob[b]) / np.sqrt(neig2*b_prob[b]*(1-b_prob[b]))
                    max_b = max(max_b, stat_b)
            if max_b <= tp_thr:
                tp_list.append(True)
            else:
                tp_list.append(False)
        else:
            tp_list.append(False)

    pos_rate = sum(p_list)/len(p_list)
    truepos_rate = sum(tp_list)/len(tp_list)
    return pos_rate, truepos_rate


In [24]:
import pandas as pd
import scanpy as sc

def evaluate_embeddings(adata, file_paths, embedding_keys, batch_key='batch', celltype_key='cell_type', use_raw=False, k1=20, k2=100, tp_thr=3.0, distance='cosine'):
    results = []  # Initialize results as a list
    
    for file_path in file_paths:
        # Read the CSV file into a DataFrame
        combined_embeddings = pd.read_csv(file_path, index_col=0)
        
        # Ensure indices match
        combined_embeddings = combined_embeddings.loc[adata.obs_names]
        
        # Split the DataFrame into separate DataFrames for each embedding
        embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}
        
        # Assign the embeddings back to adata.obsm
        for key, df in embeddings_dict.items():
            adata.obsm[key] = df.values
            
            # Evaluate embedding
            pos_rate, truepos_rate = positive_true_positive(
                adata,
                batch_key=batch_key,
                celltype_key=celltype_key,
                use_raw=use_raw,
                k1=k1,
                k2=k2,
                tp_thr=tp_thr,
                distance=distance,
                embed=key
            )
            
            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'pos_rate': pos_rate,
                'truepos_rate': truepos_rate
            }
            results.append(result_entry)
            
            # Print the file path, embedding key, and result
            print(f"File Path: {file_path}, Embedding Key: {key}, pos_rate: {pos_rate}, truepos_rate: {truepos_rate}")
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [25]:
file_paths = [
    # "../embeddings/full_annotated_supervised_human_immune_new.csv",
    # "../embeddings/cell_assign_human_immune_new.csv",
    # "../embeddings/azimuth_human_immune_new.csv",
    # "../embeddings/singleR_human_immune_new.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_30.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_50.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_70.csv",
    # "../embeddings/mixing_at_edge_human_immune_new_30.csv",
    # "../embeddings/mixing_at_edge_human_immune_new_50.csv",
    # "../embeddings/mixing_at_edge_human_immune_new_70.csv",
    # "../embeddings/randomly_missing_human_immune_new_30.csv",
    # "../embeddings/randomly_missing_human_immune_new_50.csv",
    # "../embeddings/randomly_missing_human_immune_new_70.csv"
    "../embeddings/azimuth_fuzzy_human_immune_new.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_ItClust"]


results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

# Save the DataFrame to a CSV file
# results_df.to_csv('../metrics/human_immune_true_positive_true_scores_new.csv', index=False)
results_df

File Path: ../embeddings/azimuth_fuzzy_human_immune_new.csv, Embedding Key: X_ItClust, pos_rate: 0.8154957321076822, truepos_rate: 0.31839073598758433


,file_path,embedding_key,pos_rate,truepos_rate
0,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,0.815496,0.318391


In [14]:
results_df

,file_path,embedding_key,pos_rate,truepos_rate
0,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_scDREAMER,0.890766,0.611055
1,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,0.863010,0.229004


## Print the overall metrics for kBET

In [8]:
import pandas as pd
import scanpy as sc
import numpy as np
import scib  
import scipy.sparse as sp

# teach all csr_matrix objects that `.A` means “dense array”
sp.csr_matrix.A = property(lambda self: self.toarray())

def evaluate_embeddings(adata, file_paths, embedding_keys, batch_key='batch', celltype_key='cell_type'):
    results = []  # Initialize results as a list

    for file_path in file_paths:
        # Read the CSV file into a DataFrame
        combined_embeddings = pd.read_csv(file_path, index_col=0)

        # Ensure indices match
        combined_embeddings = combined_embeddings.loc[adata.obs_names]

        # Split the DataFrame into separate DataFrames for each embedding
        embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}

        # Assign the embeddings back to adata.obsm
        for key, df in embeddings_dict.items():
            adata.obsm[key] = df.values

            # Calculate kBET score
            kbet_score = scib.metrics.kBET(
                adata,
                batch_key=batch_key,
                label_key=celltype_key,
                type_=None,
                embed=key,
                scaled=True,
                verbose=False,
            )

            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'kbet_score': kbet_score
            }
            results.append(result_entry)

            # Print the file path, embedding key, and result
            print(f"File Path: {file_path}, Embedding Key: {key}, kBET Score: {kbet_score}")

    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)

    return results_df

In [9]:
# Assuming adata is your Annotated Data object
import numpy as np
from scipy.sparse import csr_matrix

file_paths = [
    "../embeddings/randomly_wrong_human_immune_30.csv",
    "../embeddings/randomly_wrong_human_immune_50.csv",
    "../embeddings/randomly_wrong_human_immune_70.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_scANVI", "X_scgen", "X_stacas", "X_scDREAMER", "X_itclust"]

results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 6
Adding diffusion to step 4
File Path: ../embeddings/randomly_wrong_human_immune_30.csv, Embedding Key: X_scANVI, kBET Score: 0.2897136746147442
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
File Path: ../embeddings/randomly_wrong_human_immune_30.csv, Embedding Key: X_scgen, kBET Score: 0.28741490813334747
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 6
Adding diffusion to step 4
Adding diffusion to step 4
Adding diffusion to step 4
File Path: ../embeddings/randomly_wrong_human_immune_30.csv, Embedding Key: X_stacas, kBET Score: 0.24446770805691553
Adding diffusion to step 4


In [10]:
# Save the DataFrame to a CSV file
results_df.to_csv('./raw_results/human_immune_kbet_randomly_wrong.csv', index=False)

In [11]:
results_df

,file_path,embedding_key,kbet_score
0,../embeddings/randomly_wrong_human_immune_30.csv,X_scANVI,0.289714
1,../embeddings/randomly_wrong_human_immune_30.csv,X_scgen,0.287415
2,../embeddings/randomly_wrong_human_immune_30.csv,X_stacas,0.244468
3,../embeddings/randomly_wrong_human_immune_30.csv,X_scDREAMER,0.611706
4,../embeddings/randomly_wrong_human_immune_30.csv,X_itclust,0.304314
5,../embeddings/randomly_wrong_human_immune_50.csv,X_scANVI,0.305417
6,../embeddings/randomly_wrong_human_immune_50.csv,X_scgen,0.195269
7,../embeddings/randomly_wrong_human_immune_50.csv,X_stacas,0.252306
8,../embeddings/randomly_wrong_human_immune_50.csv,X_scDREAMER,0.674583
9,../embeddings/randomly_wrong_human_immune_50.csv,X_itclust,0.080453


In [8]:
file_paths = [
    "../embeddings/full_annotated_supervised_human_immune_new.csv",
    "../embeddings/cell_assign_human_immune_new.csv",
    "../embeddings/azimuth_human_immune_new.csv",
    "../embeddings/singleR_human_immune_new.csv",
    "../embeddings/partially_annotated_batches_human_immune_new_30.csv",
    "../embeddings/partially_annotated_batches_human_immune_new_50.csv",
    "../embeddings/partially_annotated_batches_human_immune_new_70.csv",
    "../embeddings/mixing_at_edge_human_immune_new_30.csv",
    "../embeddings/mixing_at_edge_human_immune_new_50.csv",
    "../embeddings/mixing_at_edge_human_immune_new_70.csv",
    "../embeddings/randomly_missing_human_immune_new_30.csv",
    "../embeddings/randomly_missing_human_immune_new_50.csv",
    "../embeddings/randomly_missing_human_immune_new_70.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_scDREAMER", "X_ItClust"]


results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

# Save the DataFrame to a CSV file
results_df.to_csv('../metrics/human_immune_kbet_scores_new.csv', index=False)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/Users/sheena/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:229: DeprecationWarning: The global conve

File Path: ../embeddings/full_annotated_supervised_human_immune_new.csv, Embedding Key: X_scDREAMER, kBET Score: 0.6432406977945416


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:158: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)


Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 6


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:158: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)


...

Adding diffusion to step 4
Adding diffusion to step 5


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:158: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  comp_size = pd.value_counts(labs)


Adding diffusion to step 4
Adding diffusion to step 5


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/kbet.py:229: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


File Path: ../embeddings/randomly_missing_human_immune_new_70.csv, Embedding Key: X_ItClust, kBET Score: 0.37436668911133975


## Print the overall metrics for LISI

In [26]:
# LISI
def calculate_LISI(adata, labels=None, total_cells=None, batch_key='batch', celltype_key='celltype', embed = 'X_pca'):
    if celltype_key is None:
        # Calculate bLISI
        lisi_b = 0
        lisi_c = lisi_f1 = np.nan
        if adata.shape[0] < 90:
            perplexity = int(adata.shape[0]/6)
        else:
            perplexity = 30
        lisi_res = lisi(adata.obsm[embed], adata.obs, batch_key, perplexity=perplexity)
        lisi_b = (np.mean(lisi_res)[batch_key]-1.)/(len(set(adata.obs[batch_key]))-1.)
    else:
        # Calculate 1-cLISI
        lisi_res = lisi(adata.obsm[embed], adata.obs, celltype_key)
        lisi_c = 1 - (np.mean(lisi_res)-1.)/(len(set(adata.obs[celltype_key]))-1.)

        # Calculate bLISI
        lisi_b = 0
        for label in labels:
            adata_sub = adata[adata.obs[celltype_key] == label]
            if adata_sub.shape[0] < 90:
                perplexity = int(adata_sub.shape[0]/6)
            else:
                perplexity = 30
            lisi_res = lisi(adata_sub.obsm[embed], adata_sub.obs, batch_key, perplexity=perplexity)
            lisi_batch = (np.mean(lisi_res)-1.)/(len(set(adata_sub.obs[batch_key]))-1.)
            lisi_b += lisi_batch*adata_sub.shape[0]
        lisi_b /= total_cells
        lisi_c = lisi_c.item()
        lisi_b = lisi_b.item()  # This will print only the value without the data type

        # Calcualte F1 score
        lisi_f1 = (2*lisi_c*lisi_b)/(lisi_c + lisi_b)
    
    return lisi_c, lisi_b, lisi_f1

In [27]:
def evaluate_embeddings(adata, file_paths, embedding_keys, batch_key='batch', celltype_key='cell_type'):
    results = []  # Initialize results as a list
    
    for file_path in file_paths:
        # Read the CSV file into a DataFrame
        combined_embeddings = pd.read_csv(file_path, index_col=0)
        
        # Ensure indices match
        combined_embeddings = combined_embeddings.loc[adata.obs_names]
        
        # Split the DataFrame into separate DataFrames for each embedding
        embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}
        
        # Assign the embeddings back to adata.obsm
        for key, df in embeddings_dict.items():
            adata.obsm[key] = df.values
            
            # Evaluate embedding
            lisi_c, lisi_b, lisi_f1 = calculate_LISI(
                adata, labels=adata.obs[celltype_key].unique(), total_cells=adata.shape[0], batch_key=batch_key, celltype_key=celltype_key, embed=key
            )
            
            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'lisi_c': lisi_c,
                'lisi_b': lisi_b,
                'lisi_f1': lisi_f1
            }
            results.append(result_entry)
            
            # Print the file path, embedding key, and result
            print(f"File Path: {file_path}, Embedding Key: {key}, LISI_c: {lisi_c}, LISI_b: {lisi_b}, LISI_f1: {lisi_f1}")
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [28]:
file_paths = [
    # "../embeddings/full_annotated_supervised_human_immune_new.csv",
    # "../embeddings/cell_assign_human_immune_new.csv",
    # "../embeddings/azimuth_human_immune_new.csv",
    # "../embeddings/singleR_human_immune_new.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_30.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_50.csv",
    # "../embeddings/partially_annotated_batches_human_immune_new_70.csv",
    # "../embeddings/mixing_at_edge_human_immune_new_30.csv",
    # "../embeddings/mixing_at_edge_human_immune_new_50.csv",
    # "../embeddings/mixing_at_edge_human_immune_new_70.csv",
    # "../embeddings/randomly_missing_human_immune_new_30.csv",
    # "../embeddings/randomly_missing_human_immune_new_50.csv",
    # "../embeddings/randomly_missing_human_immune_new_70.csv"
    "../embeddings/azimuth_fuzzy_human_immune_new.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_ItClust"]

results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

# Save the DataFrame to a CSV file
# results_df.to_csv('../metrics/human_immune_lisi_scores_new.csv', index=False)

File Path: ../embeddings/azimuth_fuzzy_human_immune_new.csv, Embedding Key: X_ItClust, LISI_c: 0.9748446639302185, LISI_b: 0.30837692328128347, LISI_f1: 0.4685388729989143


In [29]:
results_df

,file_path,embedding_key,lisi_c,lisi_b,lisi_f1
0,../embeddings/azimuth_fuzzy_human_immune_new.csv,X_ItClust,0.974845,0.308377,0.468539


# Combine the results for each settings

In [2]:
import pandas as pd

# 读入Excel文件
file_path = './human_immune_metrics.csv'  # 替换为你的文件路径
df = pd.read_csv(file_path)

# 处理数据，对每一个setting生成一个表
settings = df['type'].unique()
output_file = pd.ExcelWriter('./human_immune_metrics_grouped.xlsx')

# 创建一个Excel writer对象，利用openpyxl引擎
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for setting in settings:
        setting_df = df[df['type'] == setting]

        # 创建透视表，将行设为method，列设为metric，score为值
        pivot_table = setting_df.pivot(index='method', columns='metric', values='score')

        # 写入新的Excel sheet中
        pivot_table.to_excel(writer, sheet_name=setting)